# Data Download and Exploration

This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [1]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_pums_2017_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

In [3]:
from src.data import data_collection

This line may take as long as 10-20 minutes depending on your network connection and computer specs

In [4]:
data_collection.download_data_and_load_into_sql()

DuplicateDatabase: database "opportunity_youth" already exists


Now it's time to explore the data!

In [5]:
import psycopg2
import pandas as pd

In [6]:
DBNAME = "opportunity_youth"

In [7]:
conn = psycopg2.connect(dbname=DBNAME)

In [8]:
pd.read_sql("SELECT * FROM pums_2017 LIMIT 10;", conn)

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000000006,9,01,11606,4,53,1061971,27.0,68.0,...,53.0,24.0,39.0,24.0,7.0,27.0,8.0,46.0,25.0,50.0
1,P,2013000000006,9,02,11606,4,53,1061971,22.0,66.0,...,49.0,21.0,38.0,20.0,7.0,25.0,8.0,41.0,22.0,47.0
2,P,2013000000012,9,01,10100,4,53,1061971,22.0,72.0,...,24.0,22.0,25.0,7.0,21.0,35.0,6.0,22.0,6.0,37.0
3,P,2013000000012,9,02,10100,4,53,1061971,19.0,64.0,...,21.0,18.0,19.0,7.0,17.0,29.0,6.0,19.0,6.0,29.0
4,P,2013000000038,9,01,11505,4,53,1061971,4.0,52.0,...,4.0,1.0,2.0,8.0,8.0,1.0,4.0,6.0,1.0,4.0
5,P,2013000000038,9,02,11505,4,53,1061971,4.0,51.0,...,4.0,1.0,1.0,8.0,7.0,1.0,4.0,7.0,2.0,4.0
6,P,2013000000038,9,03,11505,4,53,1061971,7.0,18.0,...,5.0,2.0,3.0,14.0,12.0,2.0,8.0,13.0,3.0,6.0
7,P,2013000000070,9,01,10400,4,53,1061971,15.0,59.0,...,26.0,14.0,14.0,26.0,15.0,16.0,33.0,16.0,15.0,15.0
8,P,2013000000070,9,02,10400,4,53,1061971,18.0,56.0,...,27.0,15.0,16.0,30.0,14.0,18.0,38.0,17.0,17.0,14.0
9,P,2013000000082,9,01,11615,4,53,1061971,90.0,40.0,...,119.0,152.0,33.0,30.0,100.0,77.0,115.0,85.0,114.0,29.0


Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

In [9]:
('11610', '11611', '11612', '11613', '11614', '11615')

('11610', '11611', '11612', '11613', '11614', '11615')

In [10]:
!pip install folium

In [15]:
import folium
import geopandas as gpd

In [16]:
coordinates = [47.5480, -121.9836]
# 47.5480° N, 121.9836° W
kc_latitude = coordinates[0]
kc_longitude = coordinates[1]

In [17]:
King_County_WA_map = folium.Map([kc_latitude, kc_longitude])
King_County_WA_map

Make sure you close the DB connection when you are done using it

In [14]:
type(King_County_WA_map)

folium.folium.Map

In [28]:
zipfile = "zip:///Users/romanokonesnikov/Downloads/tl_2017_53_puma10.zip/tl_2017_53_puma10.shp"

In [30]:
df = gpd.read_file(zipfile)
df

,STATEFP10,PUMACE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,53,10200,5310200,"Skagit, Island & San Juan Counties PUMA",G6120,S,5470622131,2441658886,+48.4470835,-122.1737477,"POLYGON ((-122.44422 48.64527, -122.43685 48.6..."
1,53,10100,5310100,Whatcom County--Bellingham City PUMA,G6120,S,5459332804,1028183246,+48.8426531,-121.8364325,"POLYGON ((-122.40642 49.00230, -122.40617 49.0..."
2,53,10400,5310400,"Stevens, Okanogan, Pend Oreille & Ferry Counti...",G6120,S,29389124389,489162432,+48.5031090,-118.7866777,"POLYGON ((-118.14859 47.88667, -118.14899 47.8..."
3,53,10504,5310504,Spokane County (Outer)--Cheney City PUMA,G6120,S,3983412021,35812133,+47.5972847,-117.1210417,"POLYGON ((-117.10528 47.61018, -117.10527 47.6..."
4,53,10503,5310503,Spokane County (East Central)--Greater Spokane...,G6120,S,270926976,5004332,+47.6561281,-117.1818649,"POLYGON ((-117.15534 47.69971, -117.15509 47.6..."
5,53,10600,5310600,"Whitman, Asotin, Adams, Lincoln, Columbia & Ga...",G6120,S,22300152581,179320877,+46.9464148,-118.0223129,"POLYGON ((-117.38268 45.99804, -117.38543 45.9..."
6,53,10703,5310703,"Walla Walla, Benton (Outer) & Franklin (Outer)...",G6120,S,10401922840,248495777,+46.3118847,-119.0424324,"POLYGON ((-118.34166 46.73725, -118.33925 46.7..."
7,53,11000,5311000,"Lewis, Klickitat & Skamania Counties PUMA",G6120,S,15362252097,245302530,+46.2873462,-121.7656479,"POLYGON ((-122.14167 46.75410, -122.14165 46.7..."
8,53,11502,5311502,Pierce County (Northwest)--Peninsula Region & ...,G6120,S,350604568,243415998,+47.2822884,-122.6738855,"POLYGON ((-122.55705 47.40342, -122.55695 47.4..."
9,53,11612,5311612,"King County (Far Southwest)--Federal Way, Des ...",G6120,S,160638807,279665546,+47.4764651,-122.4271168,"POLYGON ((-122.33587 47.26382, -122.33676 47.2..."


In [31]:
type(df)

geopandas.geodataframe.GeoDataFrame

In [ ]:
conn.close()